## knock 086 経度緯度から距離を計算する

In [39]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列の表示が省略されないようにする
import polars.selectors as cs# 抽出条件のプリセット

### データを読み込む

In [40]:
df_customer = pl.read_csv("../docker/work/data/customer.csv")
display(df_customer.head())

df_geocode = pl.read_csv("../docker/work/data/geocode.csv")
display(df_geocode.head())

df_store = pl.read_csv("../docker/work/data/store.csv")
display(df_store.head())

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,i64,str,str,i64,str,str,str,i64,str
"""CS021313000114…","""大野 あや子""",1,"""女性""","""1981-04-29""",37,"""259-1113""","""神奈川県伊勢原市粟窪****…","""S14021""",20150905,"""0-00000000-0"""
"""CS037613000071…","""六角 雅彦""",9,"""不明""","""1952-04-01""",66,"""136-0076""","""東京都江東区南砂******…","""S13037""",20150414,"""0-00000000-0"""
"""CS031415000172…","""宇多田 貴美子""",1,"""女性""","""1976-10-04""",42,"""151-0053""","""東京都渋谷区代々木*****…","""S13031""",20150529,"""D-20100325-C"""
"""CS028811000001…","""堀井 かおり""",1,"""女性""","""1933-03-27""",86,"""245-0016""","""神奈川県横浜市泉区和泉町**…","""S14028""",20160115,"""0-00000000-0"""
"""CS001215000145…","""田崎 美紀""",1,"""女性""","""1995-03-29""",24,"""144-0055""","""東京都大田区仲六郷*****…","""S13001""",20170605,"""6-20090929-2"""


postal_cd,prefecture,city,town,street,address,full_address,longitude,latitude
str,str,str,str,str,str,str,f64,f64
"""060-0000""","""北海道""","""札幌市中央区""",null,null,null,"""北海道札幌市中央区""",141.34103,43.05513
"""064-0941""","""北海道""","""札幌市中央区""","""旭ケ丘""",null,null,"""北海道札幌市中央区旭ケ丘""",141.31972,43.04223
"""060-0042""","""北海道""","""札幌市中央区""","""大通西""",null,"""１丁目""","""北海道札幌市中央区大通西１丁…",141.35637,43.06102
"""060-0042""","""北海道""","""札幌市中央区""","""大通西""",null,"""２丁目""","""北海道札幌市中央区大通西２丁…",141.35445,43.0608
"""060-0042""","""北海道""","""札幌市中央区""","""大通西""",null,"""３丁目""","""北海道札幌市中央区大通西３丁…",141.35275,43.06086


store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
str,str,i64,str,str,str,str,f64,f64,f64
"""S12014""","""千草台店""",12,"""千葉県""","""千葉県千葉市稲毛区千草台一丁…","""チバケンチバシイナゲクチグサ…","""043-123-4003""",140.118,35.63559,1698.0
"""S13002""","""国分寺店""",13,"""東京都""","""東京都国分寺市本多二丁目""","""トウキョウトコクブンジシホン…","""042-123-4008""",139.4802,35.70566,1735.0
"""S14010""","""菊名店""",14,"""神奈川県""","""神奈川県横浜市港北区菊名一丁…","""カナガワケンヨコハマシコウホ…","""045-123-4032""",139.6326,35.50049,1732.0
"""S14033""","""阿久和店""",14,"""神奈川県""","""神奈川県横浜市瀬谷区阿久和西…","""カナガワケンヨコハマシセヤク…","""045-123-4043""",139.4961,35.45918,1495.0
"""S14036""","""相模原中央店""",14,"""神奈川県""","""神奈川県相模原市中央二丁目""","""カナガワケンサガミハラシチュ…","""042-123-4045""",139.3716,35.57327,1679.0


### ノック

In [41]:
import math

df_knock = (
    df_customer
    .select([
        pl.col("customer_id"),
        pl.col("address").alias("customer_address"),
        pl.col("postal_cd"),
        pl.col("application_store_cd")
    ])
    # 郵便番号ごとの経度(longitude)・緯度(latitude)の平均値を加える
    .join(
        other =
        df_geocode
        .group_by(by = "postal_cd")
        .agg([
            pl.mean("longitude").alias("longitude_customer"),
            pl.mean("latitude").alias("latitude_customer")
        ]),
        on = "postal_cd",
        how = "left"        
    )
    # 店舗の住所情報を加える
    .join(
        other = df_store
        .select([
            pl.col("store_cd"),
            pl.col("address").alias("store_address"),
            pl.col("longitude").alias("longitude_store"),
            pl.col("latitude").alias("latitude_store")
        ]),
        left_on = "application_store_cd",
        right_on = "store_cd",
        how = "left"
    )
    # ラジアン列を追加する
    .with_columns([
        (pl.col("longitude_customer") / 180 * math.pi).alias("λ1"),
        (pl.col("latitude_customer") / 180 * math.pi).alias("φ1"),
        (pl.col("longitude_store") / 180 * math.pi).alias("λ2"),
        (pl.col("latitude_store") / 180 * math.pi).alias("φ2"),
    ])
    # 簡易式に基づいて顧客住所と店舗との距離を計算する
    .with_columns(
        (6371
         * (pl.col("φ1").sin() * pl.col("φ2").sin()
            + pl.col("φ1").cos() * pl.col("φ2").cos() * (pl.col("λ1") - pl.col("λ2")).cos()
            )
            .arccos()
        )
        .alias("distance")
    )
    #
    .select([
        pl.col("customer_id"),
        pl.col("customer_address"),
        pl.col("store_address"),
        pl.col("distance")
    ])
)

# 先頭10行を表示する
df_knock.head(n = 10)

customer_id,customer_address,store_address,distance
str,str,str,f64
"""CS021313000114…","""神奈川県伊勢原市粟窪****…","""神奈川県伊勢原市伊勢原四丁目…",1.394409
"""CS037613000071…","""東京都江東区南砂******…","""東京都江東区南砂一丁目""",1.451182
"""CS031415000172…","""東京都渋谷区代々木*****…","""東京都渋谷区初台二丁目""",0.411733
"""CS028811000001…","""神奈川県横浜市泉区和泉町**…","""神奈川県横浜市瀬谷区二ツ橋町…",8.065196
"""CS001215000145…","""東京都大田区仲六郷*****…","""東京都大田区仲六郷二丁目""",1.268421
"""CS020401000016…","""東京都板橋区若木******…","""東京都北区十条仲原三丁目""",4.185905
"""CS015414000103…","""東京都江東区北砂******…","""東京都江東区南砂二丁目""",1.449673
"""CS029403000008…","""千葉県浦安市海楽******…","""千葉県浦安市東野一丁目""",0.804858
"""CS015804000004…","""東京都江東区北砂******…","""東京都江東区南砂二丁目""",1.449673


### 検算

In [43]:
df_knock.filter(
    (pl.col("customer_id") == "CS021313000114") |
    (pl.col("customer_id") == "CS021303000023") |
    (pl.col("customer_id") == "CS021303000007") |
    (pl.col("customer_id") == "CS021313000183") |
    (pl.col("customer_id") == "CS021314000098") |

    (pl.col("customer_id") == "CS021314000093") |
    (pl.col("customer_id") == "CS021413000049") |
    (pl.col("customer_id") == "CS021313000025") |
    (pl.col("customer_id") == "CS021413000022") |
    (pl.col("customer_id") == "CS021413000094")
)

customer_id,customer_address,store_address,distance
str,str,str,f64
"""CS021313000114…","""神奈川県伊勢原市粟窪****…","""神奈川県伊勢原市伊勢原四丁目…",1.394409
"""CS021313000025…","""神奈川県伊勢原市伊勢原***…","""神奈川県伊勢原市伊勢原四丁目…",0.474282
"""CS021413000022…","""神奈川県伊勢原市伊勢原***…","""神奈川県伊勢原市伊勢原四丁目…",0.474282
"""CS021413000094…","""神奈川県伊勢原市伊勢原***…","""神奈川県伊勢原市伊勢原四丁目…",0.474282
"""CS021303000023…","""神奈川県伊勢原市粟窪****…","""神奈川県伊勢原市伊勢原四丁目…",1.394409
"""CS021303000007…","""神奈川県伊勢原市粟窪****…","""神奈川県伊勢原市伊勢原四丁目…",1.394409
"""CS021313000183…","""神奈川県伊勢原市粟窪****…","""神奈川県伊勢原市伊勢原四丁目…",1.394409
"""CS021314000098…","""神奈川県伊勢原市粟窪****…","""神奈川県伊勢原市伊勢原四丁目…",1.394409
"""CS021314000093…","""神奈川県伊勢原市粟窪****…","""神奈川県伊勢原市伊勢原四丁目…",1.394409
